# iLOrest: the HPE Redfish swiss knife

Version 0.44

## What is iLOrest?

  * Command line Redfish client primarily designed to managed HPE iLO 4 and iLO 5 based servers
  * Python oriented
  * Sources on GitHub ([python-redfish-utility](https://github.com/HewlettPackard/python-redfish-utility))
  * [Packaged binaries](https://github.com/HewlettPackard/python-redfish-utility/releases/latest) available for several Operating Environments (.deb, .rpm, .msi and .pkg)
  * Windows and RPM packages available on [HPE Customer Support Center](http://hpe.com/info/resttool)

### Extreme flexibility

  * Interactive mode allows you to save and load settings from a file with command completion (tab key) and command recall (up and down arrows)
  * Script mode: easy integration with bash, PowerShell or DOS
  * File-based mode: allows you to save, customize and then deploy settings
  * Extensible (See [tutorial video](https://www.youtube.com/watch?v=CGl8DsYTdQo))
  * In-band and out-of-band management
  * Useful for debugging python and PowerShell scripts
  * Rich eco-system with [videos](https://www.youtube.com/playlist?list=PLmYBqUM74OygZjhoZMEZmMP50Od8EfaW8) and [articles](https://developer.hpe.com/blog)

## Interactive Out-of-Band mode demo

<img src="Pictures/OutOfBandManagement.png" alt="Out-Of-Band management" style="with: 125px;"/>

### Command completion and recall with help description

Open a `Terminal` from a Jupyter Launcher
<img src="Pictures/LauncherIcon.png" style="display:inline;width=55px;height=25px"/>
and issue the following commands. To ease this exercise, you can open a New View for this Notebook by right clicking on its tabulation: 
<img src="Pictures/NotebookNewView.png" style="display:inline;width=55px;height=25px"/>

```bash

# Invoke iLOrest with no arguments
ilorest 

# Type beginning of command and hit the Tab key to complete the command
ilorest > he (tab)

# Recall help command with up-arrow

# login remote iLO as user student. If password is not suppplied, iLOrest asks for password in hidden text.
iLOrest > login ilo5 -u student
password: P@ssw0rd!
``` 

### Data/Resource Types

Redfish resources are associated with a data type. The list of all iLO 5 data types can be retrieved with the `types` command. Each data type and its content is present in the [API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#resource-definitions).

```bash
# List all data types
iLOrest > types
```

Data types with prefix `Hpe` are Oem/Hpe specific types. Others are standard Redfish types. The following cell lists all data types present in the Redfish service, selects the `Bios` data type and show all the resources associated to that type.


```bash
# Select the Bios type
iLOrest > select Bio(Tab)

# View selected type
iLOrest > select

# Show resources associated to the selected type 
iLOrest > get

# Retrieve reseources from another type in json format
iLOrest > get --json --select Manager.

# Logout: close session and remove cache
iLOrest > logout
iLOrest > exit
```

### In-Band management demo: Modification of a BIOS attribute

<img src="Pictures/InBandManagement.png" alt="In-Band management" style="with: 125px;"/>

> **NOTE**: Only a privilege instructor can perform the following commands

```bash
ssh root@ilo5server

# When logged as root, no need to provide credentials
ilorest login

# Retrieve and modify Bios parameter
ilorest get AdminName --select Bios.
ilorest set AdminName="John Deuf" 
ilorest status --json    # Useful for rawpatch command

# Commit modification
ilorest commit

# Reset cache and view modification
ilorest select Bios. --refresh
ilorest get AdminName

# Logout (complete cache removal)
ilorest logout
 
```

## Script mode examples

### Environment preparation

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation ((Version: 0.2)) ############

# Set Student ID number
Stud=00

# location and ports variables
let OpenBmcPort=44000+${Stud}
let iLO5SimulatorPort=45000+${Stud}
let ilo5Port=443

CacheDir="${PWD}/iLOrestCache"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

OpenBmcIP=openbmcsimulators
OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"


ilo5IP="ilo5"
#ilo5="${ilo5IP}:${ilo5Port}"
ilo5="${ilo5IP}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
alias ilorest="ilorest --nologo --cache-dir=${CacheDir} --logdir=${LogDir}"
ilorest logout &>/dev/null # Clear cache


# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}")  &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

###  Log into a real iLO 5 to populate the cache directory

iLOrest populates a cache directory during the execution of the `login` command just after a successul authentication. However the DMTF Redfish simulator used in this workshop does not implement authentication and returns an error to the `login` command preventing iLOrest to populate its cache.

To overcome this problem you will first log into a physical iLO 5 and then hack the cache to make it point the DMTF iLO 5 simulator.

In [ ]:
ilorest login $ilo5 -u $User -p $Password
head ${CacheDir}/cache/* | grep url | sort -u                     # extract URL target in cache

# Copy cache in a safe location before logging out to
# minimize opened sessions. iLO 5 is limited to 15 concurrent
# sessions. 
mv ${CacheDir}/cache{,-bck}
ilorest logout

# Move back the cache
mv ${CacheDir}/cache{-bck,}

# Redirect cache to point to the iLO 5 Simulator
sed -i "s.${ilo5URI}.${iLO5SimulatorURI}." ${CacheDir}/cache/*
head ${CacheDir}/cache/* | grep url | sort -u                     # extract URL target in cache

## Get multiple properties with single request

The following request retrives all firmware versions and description with a single command. Note that DMTF includes computer firmware in the `SoftwareInventory` collection.

> **NOTE**: iLOrest versions 3.0, 3.1.0 and 3.1.1 return an `ERROR: list index out of range` message.

In [ ]:
ilorest --version

ilorest get Name Version --select SoftwareInventory

### Change iLO time zone

Searching for keywords `time zone` or `timezone` in the [API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/) leads to **two** properties. A BIOS and an iLO related property. We will focus on the [iLO property](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#timezone).

The iLO `TimeZone` property is associated to the `HpeiLODateTime` type. The next cells selects this data type and then modify it.

In [ ]:
# select type
ilorest select HpeiLODateTime

# Get all resources from the selected data type in stripped mode
ilorest get --json

In [ ]:
# Get all resources associated to this data type in verbose mode
ilorest list --json

In [ ]:
# You can get more info about a specific property.
# Note the "READ-ONLY" attribute.
ilorest info TimeZone

In [ ]:
# Set iLO TimeZone to "New Delhi"
ilorest set TimeZone/Name="New Delhi"

# View changed settings
ilorest status

In [ ]:
# Commit changes in Redfish server
ilorest commit

> **NOTE**: A successful `commit` against a real iLO 5 clears pending changes in the cache. However, our simulator does not behave like a real iLO 5 and we need to clean up the cache manually to mimic a real successful commit.

In [ ]:
ilorest status
ilorest select HpeiLODateTime --refresh
ilorest status

### Use of the --filter option

In this example you will use the iLOrest `--filter` option to retrieve only the IPv4 address of the iLO Dedicated Network Port.

Searching for keywords `dedicated` or `shared` in the [API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#oem-hpe-interfacetype) leads to the `EthernetInterface` data type. The following cell logs into a remote iLO 5 and retrieves all the data types containing string `Ethernet`.

In [ ]:
ilorest types | grep Ethernet

Data types with suffix [`Collection`](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#collections) group similar resources usually represented in an array of `Members` links. The following cell lists all the ethernet interfaces in a system: System Interfaces (4) and Manager Interfaces (3). 

In [ ]:
ilorest list --json --select EthernetInterfaceCollection

To retrieve the IPv4 address of the dedicated iLO network port you need to find a unique property characterizing the iLO Dedicated Network Port. The following cells select the ` EthernetInterface` data type and prints the `Name` property of each and every ethernet port in the server to verify whether it is a good candidate for a filter criteria.


In [ ]:
ilorest select EthernetInterface
ilorest list Name --json 

The output of the previous command show that Dedicated iLO Network Interface can be uniquely identified using the `Name` property. The following command displays the IPv4 configuration of the Dedicated iLO Network Interface using the `--filter` option.

In [ ]:
ilorest get IPv4Addresses --filter Name="Manager Dedicated Network Interface"

In [ ]:
ilorest get IPv4Addresses/Address --filter Name="Manager Dedicated Network Interface"

> Note: The `Oem/Hpe/InterfaceType` property could have been used as well.

In [ ]:
ilorest get IPv4Addresses/Address --filter Oem/Hpe/InterfaceType="Dedicated"

## Use of the iLOrest debug mode

This paragraph explains how the iLOrest debug mode can help you to troubleshoot your Python, Bash or PowerShell Redfish scripts. 

Imagine you have difficulties to develop a Python or Bash/cURL script modifying the system boot order in order to stop the next reboot of a server at RBSU/Bios Setup.

You can use iLOrest and its `bootorder` macro command in debug mode to understand how it perfoms this tasks and then reproduce it in your own programs.

The next cell prepares the environment: Remove any log file if any, login, modify the cache files.

In [ ]:
# Cleanup log file if any
rm ${LogFile} &> /dev/null

The help of the `bootorder` extension provides the syntax to send a One Time Boot command

In [ ]:
ilorest help bootorder | grep 'onetime'

The `bootorder` extension with no argument returns the list of all possible OneTimeBoot options

In [ ]:
ilorest bootorder

The following cell sends the desired `bootorder` command and retrieves the list of all modifies parameters. This is interesting information but you may need more details to understand exactly how to send the request to the remoter iLO 5.

In [ ]:
ilorest bootorder --onetimeboot=BiosSetup
ilorest status

The following command commits the changes in **debug mode** in the remore iLO 5.

Debug messages are redirected a log file, `stdout` and `stderr`. However `stdout` and `stderr` are discarded to `/dev/null` to keep a clean display in this noteobook.

In [ ]:
ilorest --debug commit &> /dev/null

# Cleaning manually pending changes in the cache to mimic a real iLO 5
ilorest select $(ilorest select | awk '{print $NF}') --refresh

### Debug file analysis

An `iLOrest.log` file should appear soon in your left Jupyter Sidebar. Double click on it and search for string `: PATCH` 

Analyse the PATCH request. Interesting for your debugging are:

  * PATH / target URI: `/redfish/v1/Systems/1/`
  * BODY / Payload request: `{"Boot": {"BootSourceOverrideTarget": "BiosSetup", "BootSourceOverrideEnabled": "Once"}}`
  * Response Code: `Code:204 No Content`
  * Response BODY: `BODY: None`
  
> NOTE: In this simulator environment the Response code is 204 (No Content) and thus the BODY is empty (`None`). If you send this request toward a real iLO 5, you will get a non-empty response.

## File-based mode

In this section you will study the `save`/`load` and `serverclone` commands. Note that the `iloclone` command is being deprepcated.

To deploy / load configuration files in parallel towards multiple targets, you can read this [article](https://developer.hpe.com/blog/massive-parallel-management-with-ilorest-on-linux).

### Save/load a specific data type configuration

In this section you will the `save` and `load` macro iLOrest commands to save and then deploy a specific or multiple data types.



By default, the output of the save macro command is written in the `ilorest.json` file in the current directory. This can be altered by the `-f` command option.

The following cell retrieves the `Chassis`and `ManagerNetworkProtocol` data types. After its execution, you will see file `ilorest.json` appearing in your left sidebar. It can take 10-15 seconds to appear in the left sidebar.

In [ ]:
# Note: there is NO space character between list items
ilorest save --multisave Chassis,ManagerNetworkProtocol                         

You can double click on the `ilorest.json` file to review its content. In addition to the `Chassis` and `ManagerNetworkProtocol` data types, it contains a `Comments` section with "administrative" properties.

The following cell modifies two parameters (`IndicatorLED` and `AlertMailEmail`) in the `ilorest.json` file.

In [ ]:
sed -i -e 's/\("IndicatorLED": \).*/\1"Blinking",/ ; s/\("AlertMailEmail": \).*/\1"John Deuf",/' ilorest.json

The following cell loads the modified configuration files into its cache, displays the modifications and commit them to the managed server.

In [ ]:
echo "Loading modified properties into target system"
ilorest load 

echo -e "\nPrint Status"
ilorest status

echo -e "\nCommit changes"
ilorest commit 

# Clean up manually pending changes in the cache to mimic a real iLO 5
ilorest select ManagerNetworkProtocol. --refresh
ilorest select Chassis. --refresh

### Save/load the entire configuration of a server

The `serverclone save` and `serverclone load` commands respectively create from a server or restore to a server a JSON formatted file containing most of its configuration settings (Smart Array configuration is not saved by default). The default output file is `ilorest_clone.json` but can be altered with the command `-f` parameter. Smart Array controller settings and logical drive configurations can be optionally be included for save.

By default the `serverclone save` command asks the user to provide inputs (i.e passwords) to be included in the output configuration file. If you don't want to supply them manually during the save operation you can provide the `--auto` parameter like in the following cell. A placeholder will be inserted in the output file for later edition.

The following cell retrieves the configuration parameters of a remove iLO 5 based server without the SSA parameters.

In [ ]:
# Retrieve server configuration including SSA parameters but without Bios parameters.
ilorest serverclone save --auto --nobios --ilossa

You can edit the `ilorest_clone.json` file, modify it and load it in suitable servers. The `--auto` paramter avoids manual inputs. When the entire file has been loaded, a server reset is automatically performed.

> Note: The HTTP responses of the iLO 5 simulator may not be identical to the resonses of a real iLO 5.

In [ ]:
ilorest serverclone load --auto -f ilorest_clone.json

# Clean up of pending changes in the cache to mimic a real iLO 5
ilorest select SmartStorageConfig. --refresh

## Raw commands

iLOrest allows you to get and set parameters directly in the Redfish tree using the following "raw" commands: `rawdelete`, `rawget`, `rawput`, `rawpost`, `rawhead` and `rawpatch`. 

The use of raw commands is not recommended as it assumes the resource URIs. Resource URIs can change over time as explained in this [article](https://developer.hpe.com/blog/getting-started-with-ilo-restful-api-redfish-api-conformance). However, in some specific cases or for troubleshooting they can be useful.


### Raw get

The `rawget` command fetches the content of the supplied URI. Refer to the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/) to find the location of your desired URI. Additional tools have to be used to filter specific properties. 

The following cell intends to retrieve the IPv4 address of the iLO Dedicated Network port, assuming it is the first NIC in the collection. It uses the popular JsonQuery (`jq`) utility to filter desired parameters. 

In [ ]:
# Get the Name and the IPv4 address parameters of the first iLO NIC. 
ilorest rawget "/redfish/v1/Managers/1/EthernetInterfaces/1" | jq '.Name, .IPv4Addresses[].Address' 

### Raw patch, put and post

The `rawpatch`, `rawput` and `rawpost` commands require a target location URI and a "body/workload" companion json file containing the parameters to patch, put or post. 

> **NOTE**: The body/workload file format has changed between ilorest versions 2.X and 3. Issue `ilorest help rawpatch` to get the file format suitable for your iLOrest version. 

The following cell creates a .json file containing the body of a patch request asking the next reboot to stop at RBSU. The content of this file comes from the debug paragraph studied earlier.

Then, it executes a `rawpatch` command this this file as input. As we are using an iLO 5 simulator, the response message may not be the one received by a real physical iLO 5.

> Note: The `ilorest status` command returns "No changes found" because the `rawpatch` command is not cached.

In [ ]:
cat > RawPatchFile.json << __EOF__
{
   "/redfish/v1/Systems/1":
   {
     "Boot": {
             "BootSourceOverrideTarget": "Utilities",
             "BootSourceOverrideEnabled": "Once"
             }  
   }
}
__EOF__

# Send the PATCH workload
ilorest rawpatch RawPatchFile.json 

# rawpatch bypasses the cache files. Hence ilorest will not detect any changes.
ilorest status 

# Logout
ilorest logout

## Summary

In this workshop, you discovered the HPE iLOrest command line interface and its three operational modes: interactive, scripted and file based. More examples can be studied in the next Jupyter Notebook.